In [1520]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

In [1521]:
df=pd.read_csv('train_df.csv')
df.head()

,id,label,statement,subject,speaker,speaker_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [1522]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10269 entries, 0 to 10268
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    10269 non-null  object
 1   label                 10269 non-null  object
 2   statement             10269 non-null  object
 3   subject               10269 non-null  object
 4   speaker               10269 non-null  object
 5   speaker_job_title     7367 non-null   object
 6   state_info            8058 non-null   object
 7   party_affiliation     10269 non-null  object
 8   barely_true_counts    10269 non-null  int64 
 9   false_counts          10269 non-null  int64 
 10  half_true_counts      10269 non-null  int64 
 11  mostly_true_counts    10269 non-null  int64 
 12  pants_on_fire_counts  10269 non-null  int64 
 13  context               10169 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.1+ MB


In [1523]:
df.label.value_counts()

label
half-true      2123
false          1998
mostly-true    1966
true           1683
barely-true    1657
pants-fire      842
Name: count, dtype: int64

In [1524]:
df.select_dtypes(include=['object']).head()

,id,label,statement,subject,speaker,speaker_job_title,state_info,party_affiliation,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN


In [1525]:
df['state_info'].isnull().sum()

np.int64(2211)

Logistic reg for one feature with 6 labels

In [1526]:
x = df['statement']
y = df['label']

In [1527]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1528]:
pipe1 = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))
])    #Here countvectorizer have only one column so it take first column as input for countvectorizer

In [1529]:
pipe1.fit(x_train, y_train)

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [1530]:
pipe1.score(x_test, y_test)

0.24878286270691333

In [1531]:
pipe1.score(x_train, y_train)

0.9013998782714546

The model is overfit 

Convert all labels to true,false

In [1532]:
df['label'].value_counts()

label
half-true      2123
false          1998
mostly-true    1966
true           1683
barely-true    1657
pants-fire      842
Name: count, dtype: int64

In [1533]:
df['label'] = df['label'].replace({'half-true': 'true' , 'mostly-true': "true",'barely-true': "true",'pants-fire': "false"})

In [1534]:
df['label'].value_counts()

label
true     7429
false    2840
Name: count, dtype: int64

Logistic reg with one feature and 2 labels

In [1535]:
x = df[['statement']]
y = df['label']

In [1587]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1588]:
pipe2 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [1537]:
y_train

9082    mostly-true
2254          false
59      mostly-true
3017          false
3853           true
           ...     
5734    mostly-true
5191      half-true
5390          false
860           false
7270      half-true
Name: label, Length: 8215, dtype: object

In [1589]:
pipe2.fit(x_train, y_train)

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [1590]:
pipe2.named_steps['classifier'].n_iter_

array([46], dtype=int32)

In [1591]:
pipe2.score(x_test, y_test)

0.7293086660175268

In [1592]:
pipe2.score(x_train, y_train)

0.7679853925745588

Again model is overfit

In [1542]:
df.columns

Index(['id', 'label', 'statement', 'subject', 'speaker', 'speaker_job_title',
       'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts',
       'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts',
       'context'],
      dtype='object')

In [1543]:
df['context'] = df['context'].dropna(axis=0)

Logisistic reg for 3 feature and 2 labels

In [1544]:
x = df[['statement', 'context', 'subject']]
y = df['label']


In [1545]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1546]:
preprocessor = ColumnTransformer(
    transformers=[
        ('context', OneHotEncoder(handle_unknown='ignore'), ['context', 'subject']),
        ('text', TfidfVectorizer(), 'statement')
    ]
)

In [1547]:
pipe3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])


In [1548]:
pipe3.fit(x_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('context', ...), ('text', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [1549]:
pipe3.named_steps['classifier'].n_iter_
pipe3.score(x_test, y_test)

0.7297955209347614

In [1550]:
pipe3.score(x_train, y_train)

0.8704808277541083

Random forest for 1 feature and 2labels[True,False]

Apply random forest for three features

In [1551]:
x=df['statement']
y = df['label']

In [1552]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1553]:
pipe4 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', RandomForestClassifier())
])  

In [1554]:
pipe4.fit(x_train, y_train)

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [1555]:
pipe4.score(x_test, y_test)

0.7366114897760467

In [1556]:
pipe4.score(x_train, y_train)

0.9996348143639684

For three features random forest and 2 labels

In [1557]:
x = df[['statement', 'context', 'subject']]
y = df['label']

In [1558]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1559]:
preprocessor = ColumnTransformer(
    transformers=[
        ('context', OneHotEncoder(handle_unknown='ignore'), ['context', 'subject']),
        ('text', TfidfVectorizer(), 'statement')
    ]
)

In [1560]:
pipe5 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])


In [1561]:
pipe5.fit(x_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('context', ...), ('text', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [1562]:
pipe5.score(x_test, y_test)

0.7385589094449854

Again overfit 

use for 3 features for knn and 2 labels

In [1563]:
x = df[['statement', 'context', 'subject']]
y = df['label']

In [1564]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1565]:
preprocessor = ColumnTransformer(
    transformers=[
        ('context', OneHotEncoder(handle_unknown='ignore'), ['context', 'subject']),
        ('text', TfidfVectorizer(), 'statement')
    ]
)

In [1566]:
pipe6 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])


In [1567]:
pipe6.fit(x_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('context', ...), ('text', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [1568]:
pipe6.score(x_test, y_test)

0.7030185004868549

In [1569]:
pipe6.score(x_train, y_train)

0.7807668898356664

Again overfit 

use for 3 features for naive bias and 2 labels

In [1570]:
#for naive bayes with 3 features
x = df[['statement', 'context', 'subject']]
y = df['label']

In [1571]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1572]:
preprocessor = ColumnTransformer(
    transformers=[
        ('context', OneHotEncoder(handle_unknown='ignore'), ['context', 'subject']),
        ('text', CountVectorizer(), 'statement')
    ]
)

In [1573]:
pipe7 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MultinomialNB())
])


In [1574]:
pipe7.fit(x_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('context', ...), ('text', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [1575]:
pipe7.score(x_test, y_test)

0.7288218111002921

In [1576]:
pipe7.score(x_train,y_train)

0.8803408399269629

use for 1 features for knn and 2 labels

In [1577]:
x=df['statement']
y = df['label']

In [1578]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [1579]:
pipe8 = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])  

In [1580]:
pipe8.fit(x_train, y_train)

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [1581]:
pipe8.score(x_test, y_test)

0.7166504381694255

In [1582]:
pipe8.score(x_train, y_train)

0.8467437614120511

# Conclusion:
1) As i use all algorithem to get the score as i get logistic regression is give good result.
2) As this train dataset not enough to train data so we to add more rows we add test dataset.